In [1]:
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_chroma import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
# load Document
file_path=os.path.join(os.curdir,'indian_foods.pdf')
loader=PyPDFLoader(file_path)
documents=loader.load()
documents

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2025-07-26T17:42:03+05:30', 'author': 'Sakib Malik', 'moddate': '2025-07-26T17:42:03+05:30', 'source': '.\\indian_foods.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content="Discover the Rich Flavors of India with \nIndian Food App 🍛 \nAt Indian Food App, we bring the authentic taste of India straight to your plate. Whether you're \ncraving a comforting bowl of butter chicken, a spicy serving of biryani, or a wholesome \nvegetarian thali, our menu is crafted to satisfy every food lover’s palate. \n\ud83e Why Choose Indian Food App? \nIndia is known for its diverse and vibrant food culture, with each region offering its own unique \ndishes and cooking styles. At Indian Food App, we celebrate this diversity by offering: \n\uf0b7 Authentic Indian Dishes – From North Indian classics like paneer tikka and dal \nmakhani to South Indian favorites such as masala dosa and s

In [3]:
# Split text into 500 words
splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
docs=splitter.split_documents(documents)
docs

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2025-07-26T17:42:03+05:30', 'author': 'Sakib Malik', 'moddate': '2025-07-26T17:42:03+05:30', 'source': '.\\indian_foods.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content="Discover the Rich Flavors of India with \nIndian Food App 🍛 \nAt Indian Food App, we bring the authentic taste of India straight to your plate. Whether you're"),
 Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2025-07-26T17:42:03+05:30', 'author': 'Sakib Malik', 'moddate': '2025-07-26T17:42:03+05:30', 'source': '.\\indian_foods.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='craving a comforting bowl of butter chicken, a spicy serving of biryani, or a wholesome \nvegetarian thali, our menu is crafted to satisfy every food lover’s palate. \n\ud83e Why Choose Indian Food App?'),
 Document(metadata={'producer': 'Micro

In [4]:
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")
persist_directory="./db"
os.makedirs(persist_directory,exist_ok=True)
vector_store=Chroma(
    embedding_function=embeddings,
    collection_name="indian_food",
    persist_directory=persist_directory
)

In [5]:
# senitize text
import re
def senitize_text(text):
    return re.sub(r'[\ud800-\udfff]','',text)


In [6]:
senitize_text("Sakib Malik \ud800")

'Sakib Malik '

In [7]:
# sentize docs
for doc in docs:
    doc.page_content=senitize_text(doc.page_content)

In [8]:
from uuid import uuid4
uuids=[str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs,ids=uuids)

['4fa605ea-3f61-48b9-bbcb-d103f116408b',
 '8b9e7bc2-f01c-444b-9c05-9c5e3228e3e5',
 '378bdf50-6881-4cc2-8727-bf57caf5b658',
 '64602b57-6bba-4aae-a76f-9e6c5a84a7f9',
 '2eac1905-3b45-4613-ad92-4a128cd4b939',
 'a6d8cdd5-f278-41b3-86ae-59f760a77be8',
 'b8b51d08-4307-40cc-a37d-ce0a20dae824',
 'b1a6a898-5473-40b1-ba11-392b851226fc',
 '042c4564-de30-41be-9bb6-84430b3f3517',
 '496e5378-f957-4239-8627-199caa357b81',
 '302cbded-2158-450b-b6e6-ce91f3dd4204',
 'a3412b3d-bfb1-4c51-8881-56b22c6b5d11',
 '574ad291-667b-4ba9-ac45-ff1d31a76d73',
 '9e95ff78-0eb9-4773-9773-6e104b694e34',
 '6a99af6c-00f6-4237-b4d6-1a66e0200a0a',
 '91687f32-922a-4c48-a9e1-2e02556824b0']

In [9]:
result=vector_store.similarity_search("What is shop name",k=1)

In [10]:
result[0].page_content

"🌐 Website: [YourWebsiteHere.com] \n \nIndian Food App – Bringing India's Taste to Your Table"

In [11]:
import requests
from langchain.tools import StructuredTool
from pydantic import BaseModel
from typing import List

# Create Structured Tool

BASE_URL="http://127.0.0.1:8000"

class FoodMenu(BaseModel):    
    name:str
    description:str
    price:int

class EmptyInput(BaseModel):
    pass

class FoodMenuInput(BaseModel):
    data:FoodMenu

class OrderModel(BaseModel):
    customer_name:str
    item_id:int
    quantity:int

class OrderInput(BaseModel):
    data:OrderModel

# Tools
def get_menu()->List[FoodMenu]:
    """Get menu Item of Indian Restaurant App
    this will return an menu items with price    
    """
    url=BASE_URL+"/menu"
    response=requests.get(url)    
    return response.json()

get_menu_tool=StructuredTool.from_function(
    func=get_menu,
    name="get_menu",
    description="It get the menu list",
    args_schema=EmptyInput
)


def create_menu(data:FoodMenuInput):
    """Create menu Function, you can create menu from this fuction
    
    args:
        name:str
        description:str
        price:int
    """                       
    url=BASE_URL+"/menu"
    response=requests.post(url,json=data.model_dump())    
    return response.json()    

create_menu_tool=StructuredTool.from_function(
    func=create_menu,
    name='create_menu',
    description="It create an menu",
    args_schema=FoodMenuInput
)




# Order Tool

# get list Of Orders.
def get_orders()->List[OrderModel]:
    url=BASE_URL+"/orders"
    response=requests.get(url)
    return response.json()

get_order_tool=StructuredTool.from_function(
    func=get_orders,
    name="get_orders",
    description="Get Orders",
    args_schema=EmptyInput
)
# Create Order Tool

def create_order(data:OrderModel)->OrderModel:    
    url=BASE_URL+"/orders"
    response=requests.post(url,json=data.model_dump())
    return response.json()    

create_order_tool=StructuredTool.from_function(
    func=create_order,
    name="create_order",
    description="Create Order Tool",
    args_schema=OrderInput
)

In [12]:
get_menu_tool.invoke({})

[{'name': 'Balu shahi',
  'description': 'Maida flour, yogurt, oil, sugar',
  'price': 171.0,
  'id': 1},
 {'name': 'Boondi',
  'description': 'Gram flour, ghee, sugar',
  'price': 121.0,
  'id': 2},
 {'name': 'Gajar ka halwa',
  'description': 'Carrots, milk, sugar, ghee, cashews, raisins',
  'price': 423.0,
  'id': 3},
 {'name': 'Ghevar',
  'description': 'Flour, ghee, kewra, milk, clarified butter, sugar, almonds, pistachio, saffron, green cardamom',
  'price': 359.0,
  'id': 4},
 {'name': 'Gulab jamun',
  'description': 'Milk powder, plain flour, baking powder, ghee, milk, sugar, water, rose water',
  'price': 245.0,
  'id': 5},
 {'name': 'Imarti',
  'description': 'Sugar syrup, lentil flour',
  'price': 172.0,
  'id': 6},
 {'name': 'Jalebi',
  'description': 'Maida, corn flour, baking soda, vinegar, curd, water, turmeric, saffron, cardamom',
  'price': 432.0,
  'id': 7},
 {'name': 'Kaju katli',
  'description': 'Cashews, ghee, cardamom, sugar',
  'price': 412.0,
  'id': 8},
 {'nam

In [13]:
data=FoodMenu(
  name="Verka Milk New New" ,
  description="Verka milk is a good milk, which is sale in punjab ares. this easily available in punjab shops",
  price=32
) 
create_menu_tool.invoke({'data':data})

{'name': 'Verka Milk New New',
 'description': 'Verka milk is a good milk, which is sale in punjab ares. this easily available in punjab shops',
 'price': 32.0,
 'id': 257}

In [86]:
get_order_tool.invoke({})

[{'customer_name': 'Sakib Choudhary',
  'item_id': 1,
  'quantity': 1,
  'id': 1,
  'item': {'name': 'Daal',
   'description': 'Made from ',
   'price': 300.0,
   'id': 1}},
 {'customer_name': 'Sakib Choudhary',
  'item_id': 2,
  'quantity': 1,
  'id': 3,
  'item': {'name': 'Balu shahi',
   'description': 'Maida flour, yogurt, oil, sugar',
   'price': 466.0,
   'id': 2}},
 {'customer_name': 'Sakib Choudhary',
  'item_id': 5,
  'quantity': 1,
  'id': 4,
  'item': {'name': 'Ghevar',
   'description': 'Flour, ghee, kewra, milk, clarified butter, sugar, almonds, pistachio, saffron, green cardamom',
   'price': 371.0,
   'id': 5}},
 {'customer_name': 'Sakib Choudhary',
  'item_id': 9,
  'quantity': 1,
  'id': 5,
  'item': {'name': 'Kaju katli',
   'description': 'Cashews, ghee, cardamom, sugar',
   'price': 440.0,
   'id': 9}},
 {'customer_name': 'Sakib Choudhary',
  'item_id': 10,
  'quantity': 1,
  'id': 6,
  'item': {'name': 'Kalakand',
   'description': 'Milk, cottage cheese, sugar',
  

In [98]:
data=OrderModel(
    customer_name="Sakib Choudhary",
    item_id=10,
    quantity= 1
)
result=create_order_tool.invoke({"data":data})
result

{'customer_name': 'Sakib Choudhary',
 'item_id': 10,
 'quantity': 1,
 'id': 7,
 'item': {'name': 'Kalakand',
  'description': 'Milk, cottage cheese, sugar',
  'price': 186.0,
  'id': 10}}

In [128]:
# Chatbot
llm=ChatOpenAI(model="gpt-4o")
llm_with_tools=llm.bind_tools([get_menu_tool,create_menu_tool,get_order_tool,create_order_tool])


In [129]:
from langchain.schema.messages import HumanMessage,AIMessage,ToolMessage
messages=[]
query=HumanMessage(content="Hii")
messages.append(query)
messages

[HumanMessage(content='Hii', additional_kwargs={}, response_metadata={})]

In [130]:
ai_res=llm_with_tools.invoke(messages)
messages.append(ai_res)
ai_res

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 113, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxuSVtjh61krxa2p9RuwtUabLcdgR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--263be36a-e86c-4553-90d3-31b389f67669-0', usage_metadata={'input_tokens': 113, 'output_tokens': 10, 'total_tokens': 123, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [131]:
query=HumanMessage(content="Can you give me list of menu items?")
messages.append(query)
ai_res=llm_with_tools.invoke(messages)
messages.append(ai_res)
ai_res

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EwU8KoGCak300QvvX5IWohG0', 'function': {'arguments': '{}', 'name': 'get_menu'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 139, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxuSXS3JQyh8yLqVNrRCxKrVyPnNq', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b9712dc4-fd5a-456c-ab8f-a4889bab7183-0', tool_calls=[{'name': 'get_menu', 'args': {}, 'id': 'call_EwU8KoGCak300QvvX5IWohG0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 10, 'total_tokens': 149, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [132]:
ai_res.tool_calls[0]
tool_res=get_menu_tool.invoke(ai_res.tool_calls[0]['args'])
tool_res

[{'name': 'Daal', 'description': 'Made from ', 'price': 300.0, 'id': 1},
 {'name': 'Balu shahi',
  'description': 'Maida flour, yogurt, oil, sugar',
  'price': 466.0,
  'id': 2},
 {'name': 'Boondi',
  'description': 'Gram flour, ghee, sugar',
  'price': 372.0,
  'id': 3},
 {'name': 'Gajar ka halwa',
  'description': 'Carrots, milk, sugar, ghee, cashews, raisins',
  'price': 203.0,
  'id': 4},
 {'name': 'Ghevar',
  'description': 'Flour, ghee, kewra, milk, clarified butter, sugar, almonds, pistachio, saffron, green cardamom',
  'price': 371.0,
  'id': 5},
 {'name': 'Gulab jamun',
  'description': 'Milk powder, plain flour, baking powder, ghee, milk, sugar, water, rose water',
  'price': 99.0,
  'id': 6},
 {'name': 'Imarti',
  'description': 'Sugar syrup, lentil flour',
  'price': 282.0,
  'id': 7},
 {'name': 'Jalebi',
  'description': 'Maida, corn flour, baking soda, vinegar, curd, water, turmeric, saffron, cardamom',
  'price': 465.0,
  'id': 8},
 {'name': 'Kaju katli',
  'description'

In [133]:
import json
tool_msg=ToolMessage(content=json.dumps(tool_res),tool_call_id=ai_res.tool_calls[0]['id'])
messages.append(tool_msg)
messages


[HumanMessage(content='Hii', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 113, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxuSVtjh61krxa2p9RuwtUabLcdgR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--263be36a-e86c-4553-90d3-31b389f67669-0', usage_metadata={'input_tokens': 113, 'output_tokens': 10, 'total_tokens': 123, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='Can you give me list of menu items?', additional_kwargs={}, response

In [135]:
messages
result=llm_with_tools.invoke(messages)
result.content

"Here's a list of some menu items available:\n\n1. **Daal** - ₹300\n   - Description: Made from lentils.\n   \n2. **Balu Shahi** - ₹466\n   - Description: Made from maida flour, yogurt, oil, and sugar.\n\n3. **Boondi** - ₹372\n   - Description: Made from gram flour, ghee, and sugar.\n\n4. **Gajar ka Halwa** - ₹203\n   - Description: Made with carrots, milk, sugar, ghee, cashews, and raisins.\n\n5. **Ghevar** - ₹371\n   - Description: Includes flour, ghee, kewra, milk, clarified butter, sugar, almonds, pistachio, saffron, green cardamom.\n\n6. **Gulab Jamun** - ₹99\n   - Description: Made from milk powder, plain flour, baking powder, ghee, milk, sugar, water, and rose water.\n\n7. **Imarti** - ₹282\n   - Description: Made with sugar syrup and lentil flour.\n\n8. **Jalebi** - ₹465\n   - Description: Made from maida, corn flour, baking soda, vinegar, curd, water, turmeric, saffron, and cardamom.\n\n9. **Kaju Katli** - ₹440\n   - Description: Made with cashews, ghee, cardamom, and sugar.\n

In [136]:
query=HumanMessage(content="Can you book an order Daal qty 1 and")
messages.append(query)
ai_res=llm_with_tools.invoke(messages)
ai_res

AIMessage(content='Please let me know the customer\'s name to proceed with the order for "Daal" with a quantity of 1.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 10793, 'total_tokens': 10818, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 10752}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxuUhimEJ3yOkotKFZ71N5MSFdJaH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0d875301-6c3d-4f0d-ab64-694c679fd9b6-0', usage_metadata={'input_tokens': 10793, 'output_tokens': 25, 'total_tokens': 10818, 'input_token_details': {'audio': 0, 'cache_read': 10752}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [137]:
messages.append(ai_res)
query=HumanMessage(content="Customer name is Sakib Malik")
messages.append(query)
ai_res=llm_with_tools.invoke(messages)
ai_res

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pzalVYRCoWbfEcXbCf8NU6p2', 'function': {'arguments': '{"data":{"customer_name":"Sakib Malik","item_id":1,"quantity":1}}', 'name': 'create_order'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 10831, 'total_tokens': 10860, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 10752}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxuVo1QO5EGZtoE7sV06jYx02j0mL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8955148d-8fed-4854-9b39-ee6af0bc3f13-0', tool_calls=[{'name': 'create_order', 'args': {'data': {'customer_name': 'Sakib Malik', 'item_id': 1, 'quantity': 1}}, 'id': 'call_pzalVYRCoWbfEcXbCf8NU6p2', 'type': 'tool_call'}]

In [138]:
messages.append(ai_res)

In [141]:
data=ai_res.tool_calls[0]['args']
id=ai_res.tool_calls[0]['id']
tool_res=create_order_tool.invoke(data)
tool_res

{'customer_name': 'Sakib Malik',
 'item_id': 1,
 'quantity': 1,
 'id': 8,
 'item': {'name': 'Daal',
  'description': 'Made from ',
  'price': 300.0,
  'id': 1}}

In [142]:
tool_r=ToolMessage(content=json.dumps(tool_res),tool_call_id=id)
messages.append(tool_r)
ai_res=llm_with_tools.invoke(messages)
ai_res

AIMessage(content='The order for "Daal" with a quantity of 1 has been successfully placed for the customer Sakib Malik. If you need further assistance, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 10928, 'total_tokens': 10964, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 10752}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxuYU8r8k14lwGUgmiMCnt5PWONqb', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e9e3ddf6-268d-436b-956c-0c98dde7666e-0', usage_metadata={'input_tokens': 10928, 'output_tokens': 36, 'total_tokens': 10964, 'input_token_details': {'audio': 0, 'cache_read': 10752}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [143]:
ai_res.content

'The order for "Daal" with a quantity of 1 has been successfully placed for the customer Sakib Malik. If you need further assistance, feel free to ask!'

In [151]:
messages.append(ai_res)
query=HumanMessage(content="Thanku You for now.")
messages.append(query)
ai_res=llm_with_tools.invoke(messages)
messages.append(ai_res)
ai_res

AIMessage(content="You're welcome! If you need assistance in the future, feel free to reach out. Have a great day!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 10990, 'total_tokens': 11013, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 10880}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxudmE1pijtmhGfrZDLTWn2Z8Bba7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--13dfe508-7c7f-48b7-877b-1e260a770eb7-0', usage_metadata={'input_tokens': 10990, 'output_tokens': 23, 'total_tokens': 11013, 'input_token_details': {'audio': 0, 'cache_read': 10880}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [152]:
messages.append(ai_res)

In [156]:
messages

[HumanMessage(content='Hii', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 113, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BxuSVtjh61krxa2p9RuwtUabLcdgR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--263be36a-e86c-4553-90d3-31b389f67669-0', usage_metadata={'input_tokens': 113, 'output_tokens': 10, 'total_tokens': 123, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='Can you give me list of menu items?', additional_kwargs={}, response

In [31]:
embeddings=OpenAIEmbeddings(model="text-embedding-3-large")
vector_store=Chroma(
    embedding_function=embeddings,
    collection_name="indian_food",
    persist_directory="./db"
)
vector_store.similarity_search('what is shop name',k=1)


[Document(id='25c56b9e-de3c-43e6-a3e9-50c3ec38909d', metadata={'page_label': '2', 'author': 'Sakib Malik', 'moddate': '2025-07-26T17:42:03+05:30', 'creationdate': '2025-07-26T17:42:03+05:30', 'producer': 'Microsoft® Word 2013', 'total_pages': 2, 'creator': 'Microsoft® Word 2013', 'page': 1, 'source': '.\\indian_foods.pdf'}, page_content="🌐 Website: [YourWebsiteHere.com] \n \nIndian Food App – Bringing India's Taste to Your Table")]